In [187]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import astropy.units as u
from astropy.constants import G
G = G.to(u.pc * u.Msun**-1 * (u.km / u.s)**2)
from astropy.coordinates import CartesianRepresentation, CartesianDifferential

import gala.potential as gp
import gala.dynamics as gd
import gala.integrate as gi
import gala.units as gu

from gala.units import galactic
from gala.potential import NFWPotential
from gala.dynamics import PhaseSpacePosition, MockStream
from gala.integrate import LeapfrogIntegrator


In [188]:
# Parameters

mass_halo = 1e12 * u.Msun
r_s = 10 * u.kpc

mass_plummer = 1e8 * u.Msun
r_plummer = 1 * u.kpc

time = 1 * u.Gyr
dt   = 1 * u.Myr

pos_p = [-50, 0, 1] * u.kpc
vel_p = [0, 175, 0] * u.km/u.s

N  = 10
dN = 1

# Define Main Halo Potential
pot_NFW = gp.NFWPotential(mass_halo, r_s, a=1, b=1, c=1, units=galactic, origin=None, R=None)

step = int(time/dt)
step_N = int(step/N)

orbit_pos_p = np.zeros((step, 3)) * u.kpc
orbit_vel_p = np.zeros((step, 3)) 
orbit_pos_p[0] = pos_p
orbit_vel_p[0] = vel_p

pos_N = np.zeros((N, 3)) * u.kpc
vel_N = np.zeros((N, 3)) * u.km/u.s

orbit_pos_N = np.zeros((step, N, 3)) * u.kpc
orbit_vel_N = np.zeros((step, N, 3)) * u.km/u.s

In [190]:
np.random.normal(vel_p, 3 * u.km/u.s)

array([ -0.74652172, 176.56551666,  -0.92377883])

In [192]:
vel_p + 3 * u.km/u.s

<Quantity [  3., 178.,   3.] km / s>

In [3]:
for i in tqdm(range(step-1)):

    if i==0: #i % step_N == 0:
        j = i//step_N

        # New N initial conditions
        pos_N[j] = pos_p # + 2 * u.kpc # tidal radius
        vel_N[j] = vel_p # velocity dispersion

    if i == 0:
        orbit_pos_N[i] = pos_N
        orbit_vel_N[i] = vel_N

    # All N in Phase Space Position
    wN = gd.PhaseSpacePosition(pos = pos_N[:j + 1],
                               vel = vel_N[:j + 1])

    # Define Plummer Potential
    pot_plummer  = gp.PlummerPotential(mass_plummer, r_plummer, units=galactic, origin=pos_p, R=None)
    pot_combined = pot_NFW + pot_plummer
    orbit_N = gp.Hamiltonian(pot_NFW).integrate_orbit(wN, dt=dt, n_steps=1)
    pos_N[:j+1] = orbit_N.xyz[:, -1]
    vel_N[:j+1] = orbit_N.v_xyz[:, -1]

    # Progenitor Phase Space Position
    wp = gd.PhaseSpacePosition(pos = pos_p,
                               vel = vel_p)
    
    # Progenitor new Position and Velocity
    orbit_p = gp.Hamiltonian(pot_NFW).integrate_orbit(wp, dt=dt, n_steps=1)
    pos_p = orbit_p.xyz[:, -1]
    vel_p = orbit_p.v_xyz[:, -1]

    # Save Progenitor new Position and Velocity
    orbit_pos_p[i+1] = pos_p
    orbit_vel_p[i+1] = vel_p

    # Save N new Position and Velocity
    orbit_pos_N[i+1] = pos_N
    orbit_vel_N[i+1] = vel_N


  0%|          | 0/999 [00:00<?, ?it/s]

: 

: 

In [151]:
import sympy as sp

def get_Jacobian(a,b,c):
    # Define the symbols for Cartesian and Spherical coordinates
    x, y, z = sp.symbols('x y z')
    r, theta, phi = sp.symbols('r theta phi')

    # Define the transformations from Cartesian to Spherical coordinates
    r_expr = sp.sqrt(x**2 + y**2 + z**2)
    theta_expr = sp.acos(z / sp.sqrt(x**2 + y**2 + z**2))
    phi_expr = sp.atan2(y, x)

    # Create the Jacobian matrix
    J = sp.Matrix([
        [r_expr.diff(x), r_expr.diff(y), r_expr.diff(z)],
        [theta_expr.diff(x), theta_expr.diff(y), theta_expr.diff(z)],
        [phi_expr.diff(x), phi_expr.diff(y), phi_expr.diff(z)]
    ])

    # Define a specific point (x, y, z)
    point = {x: a, y: b, z: c}  # Example point

    # Substitute the point into the Jacobian matrix to get numeric values
    J_numeric = J.subs(point)

    return np.array(J_numeric, dtype=float)

def get_rt(wp, pot_NFW, mass_plummer):

    rp = np.linalg.norm( wp.xyz )
    angular_velocity = ( np.linalg.norm( wp.angular_momentum() ) / rp**2 ).to(u.Gyr**-1)

    J = get_Jacobian(wp.x.value, wp.y.value, wp.z.value)
    d2pdr2 = (J.T * pot_NFW.hessian( wp )[:,:,0] * J)[0,0]
    print(d2pdr2.to(u.Gyr**-2))
    rt = ( G * mass_plummer / (angular_velocity**2 - d2pdr2) ).to(u.kpc**3) **(1/3)
    return rt


In [154]:
rp = np.linalg.norm( wp.xyz )
angular_velocity = ( np.linalg.norm( wp.angular_momentum() ) / rp**2 ).to(u.Gyr**-1)

J = get_Jacobian(wp.x.value, wp.y.value, wp.z.value)
hessian = pot_NFW.hessian( wp )[:,:,0]
d2pdr2 = (J.T * hessian * J)[0,0]
rt = ( G * mass_plummer / (angular_velocity**2 - d2pdr2) ).to(u.kpc**3) **(1/3)

In [186]:
time = 4 * u.Gyr
dt   = 10 * u.Myr

int(time/dt)

400

In [173]:
(1/rp - wp.x**2/rp**3)

<Quantity 7.9952024e-06 1 / kpc>

In [175]:
 a_vector[0]*(1/rp - wp.x**2/rp**3)**-1

<Quantity [121.14000259] kpc2 / Myr2>

In [176]:
hessian[0,0]*(wp.x/rp)**-2 

<Quantity -2.03814865e-05 1 / Myr2>

In [179]:
(hessian[0,0]*(wp.x/rp)**-2 + a_vector[0]*(1/wp.x - rp**2/wp.x**3)).to(u.Gyr**-2)

<Quantity [-20.3737382] 1 / Gyr2>

In [184]:
(hessian[2,2]*(wp.z/rp)**-2 + a_vector[2]*(1/wp.z - rp**2/wp.z**3)).to(u.Gyr**-2)

<Quantity [96833.49469795] 1 / Gyr2>

In [183]:
wp.z

<Quantity 1. kpc>

In [161]:
(1/rp - wp.x**2/rp**3)**-1

<Quantity 125075.00749952 kpc>

In [152]:
rt = get_rt(wp, pot_NFW, mass_plummer)
print(rt)

-20.36519109498713 1 / Gyr2
11.068661700563187 kpc
